书中使用的应该是`Tensorflow 1.x`系列，我安装的版本是`Tensorflow1.15.0`，`cpu版本`

## 我的测试代码

自己使用 `Tensorflow1.15.0` 时的一些测试验证代码，毕竟不知道书中用的是什么版本，这个真的得吐槽一下，写书难道不说明一下自己使用的python，tensorflow版本吗？这不是coder的常识吗？

In [1]:
import tensorflow as tf

print('Tensorflow版本号  : ',tf.__version__)
print('Tensorflow安装路径: ',tf.__path__)

Tensorflow版本号  :  1.15.0
Tensorflow安装路径:  ['d:\\ProgramData\\Anaconda3\\envs\\tf1\\lib\\site-packages\\tensorflow_core\\python\\keras\\api\\_v1', 'd:\\ProgramData\\Anaconda3\\envs\\tf1\\lib\\site-packages\\tensorflow_estimator\\python\\estimator\\api\\_v1', 'd:\\ProgramData\\Anaconda3\\envs\\tf1\\lib\\site-packages\\tensorflow_core', 'd:\\ProgramData\\Anaconda3\\envs\\tf1\\lib\\site-packages\\tensorflow_core\\_api\\v1']


# TensorFlow 的基本使用方法

In [1]:
import numpy as np
import tensorflow as tf

data_x = np.array([[0,0],[0,1],[1,0],[1,1]], dtype=np.float32)
data_y = np.array([[0],[1],[1],[0]], dtype=np.float32)


# 这里将用到 placeholder 这个结构 。 它会定义一些占位变量， 
# 这些变量在定义计算图时并不会被明确指定，它的数值将在执行计算图时才被赋值，
# 所以这个结构一般被用于传入网络的数据。 我们可以从上面的输入数据中看到数据的维度为 2 
x = tf.placeholder(tf.float32, shape=[None, 2], name='input')
# 其中 shape= [None， 2］表示在定义占位符时只知道输入的第二维是 2 ，第一维的大小并不确定




w1 = tf.Variable(tf.random_uniform([2,4], -1, 1), name='w1')
b1 = tf.Variable(tf.zeros(4, dtype=np.float32), name='b1')


w2 = tf.get_variable('w2', shape=[4,1], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable('b2', initializer=tf.constant(0.0))


z1 = tf.sigmoid(tf.matmul(x, w1) + b1)  # 第1层输出
z2 = tf.sigmoid(tf.matmul(z1, w2) + b2)  # 第2层输出


y = tf.placeholder(tf.float32, shape=[None, 1], name='output')  # 定义一个输出值
loss = tf.nn.l2_loss(z2 - y )  # 比较输出值和目标值的差距

# 完成目标函数的设定后，接下来定义优化器和优化策略
opt = tf.train.GradientDescentOptimizer(0.05)  # 定义了一个最基本的梯度下降法，它的学习率是 0.05
train_op = opt.minimize(loss)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
print('data_x shap : ', data_x.shape)
print('data_y shap : ', data_y.shape)



data_x shap :  (4, 2)
data_y shap :  (4, 1)


In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for iter in range(100000):
        loss_val, _ = sess.run([loss, train_op], feed_dict={x:data_x, y:data_y})
        if iter % 10000 == 0:
            print('{}:loss={}'.format(iter, loss_val))
    graph = tf.get_default_graph()
    print('\ngraph.get_operations():\n-----------------------')
    li_operations = graph.get_operations()
    print(li_operations)

0:loss=0.5318878889083862
10000:loss=0.020341254770755768
20000:loss=0.004817883018404245
30000:loss=0.0025588541757315397
40000:loss=0.0017095129005610943
50000:loss=0.0012724973494186997
60000:loss=0.0010085290996357799
70000:loss=0.0008327396353706717
80000:loss=0.0007075909525156021
90000:loss=0.0006142731290310621

graph.get_operations():
-----------------------
[<tf.Operation 'input' type=Placeholder>, <tf.Operation 'random_uniform/shape' type=Const>, <tf.Operation 'random_uniform/min' type=Const>, <tf.Operation 'random_uniform/max' type=Const>, <tf.Operation 'random_uniform/RandomUniform' type=RandomUniform>, <tf.Operation 'random_uniform/sub' type=Sub>, <tf.Operation 'random_uniform/mul' type=Mul>, <tf.Operation 'random_uniform' type=Add>, <tf.Operation 'w1' type=VariableV2>, <tf.Operation 'w1/Assign' type=Assign>, <tf.Operation 'w1/read' type=Identity>, <tf.Operation 'zeros/shape_as_tensor' type=Const>, <tf.Operation 'zeros/Const' type=Const>, <tf.Operation 'zeros' type=Fill>,

In [5]:
for i,op in enumerate(li_operations):
    print(i, ' : ',type(op))

0  :  <class 'tensorflow.python.framework.ops.Operation'>
1  :  <class 'tensorflow.python.framework.ops.Operation'>
2  :  <class 'tensorflow.python.framework.ops.Operation'>
3  :  <class 'tensorflow.python.framework.ops.Operation'>
4  :  <class 'tensorflow.python.framework.ops.Operation'>
5  :  <class 'tensorflow.python.framework.ops.Operation'>
6  :  <class 'tensorflow.python.framework.ops.Operation'>
7  :  <class 'tensorflow.python.framework.ops.Operation'>
8  :  <class 'tensorflow.python.framework.ops.Operation'>
9  :  <class 'tensorflow.python.framework.ops.Operation'>
10  :  <class 'tensorflow.python.framework.ops.Operation'>
11  :  <class 'tensorflow.python.framework.ops.Operation'>
12  :  <class 'tensorflow.python.framework.ops.Operation'>
13  :  <class 'tensorflow.python.framework.ops.Operation'>
14  :  <class 'tensorflow.python.framework.ops.Operation'>
15  :  <class 'tensorflow.python.framework.ops.Operation'>
16  :  <class 'tensorflow.python.framework.ops.Operation'>
17  :  

# TensorFlow 原理介绍
## 创建变量的 scope

### test1

In [12]:
# 重置变量
tf.reset_default_graph()

In [13]:
import numpy as np
import tensorflow as tf

with tf.name_scope('123'):
      with tf.name_scope('456'):  # 用的是name_scope
        with tf.variable_scope('789'):  # 用的是variable_scope
            a = tf.Variable(1,name='a')  # 对name_scope 和 variable_scope 都响应
            print('a.name :  ',a.name)
            b = tf.get_variable('b',1)  # 只对 variable_scope 响应
            print('b.name :  ',b.name)

a.name :   123/456/789/a:0
b.name :   789/b:0


### test2

name_scope 重复出现时，系统会自动改名

In [14]:
# 重置变量
tf.reset_default_graph()

In [15]:
with tf.name_scope('123'):
    with tf.name_scope('456'):
        with tf.variable_scope('789'):
            a = tf.Variable(1,name='a')
            print('a.name :  ',a.name)
            b = tf.get_variable('b',1)
            print('b.name :  ',b.name)
    with tf.name_scope('456'):  # name_scope 重复出现时，系统会自动改名
        with tf.variable_scope('789'):
            c = tf.Variable(1,name='c')
            print('c.name :  ',c.name)
            d = tf.Variable(1,name='d')
            print('d.name :  ',d.name)

a.name :   123/456/789/a:0
b.name :   789/b:0
c.name :   123/456_1/789/c:0
d.name :   123/456_1/789/d:0


### test3

同一个 scope 下出现两遍 variable_scope 时， scope 的名字不会出现名称修改

In [18]:
# 重置变量
tf.reset_default_graph()

In [19]:
with tf.name_scope('123'):
    with tf.name_scope('456'):
        with tf.variable_scope('789'):
            d = tf.Variable(1,name='d')
            print('d.name :  ',d.name)
        with tf.variable_scope('789'):
            e = tf.get_variable('e',1)
            print('e.name :  ',e.name)

d.name :   123/456/789/d:0
e.name :   789/e:0


### name_scope


In [26]:
# 重置变量
tf.reset_default_graph()

In [27]:
with tf.name_scope('123'):
    with tf.name_scope(None):
        c = tf.Variable(1,name='f')
        print('c.name :  ',c.name)

c.name :   f:0


### variable_scope

实际上， variable_scope 和 name_scope 是两个有点相互独立的命名体系， 

- name_scope只对通过 Variable 创建的变量有效;
- 而 variable_scope 还可以为 get_variable 这个可以复用的变量的方法服务;


它首先调用了 name_scope 方法，然后在 name_scop巳的 context 中调用了另外一个生成 variable_scope 的方法。

我们将整体过程分成五步：

1. 获取default_varscope
2. 获取var _store
3. 组合新 scope 的名字
4. 创建新的 varscope 并返回
5. 回收 scope 信息


### 相同点和不同点

总体来看，两种 scope 存在一些相同和不同，让我们来总结一下 。
1. 两种 scope 形成的命名空间将在 Variable 创建的变量上产生影响 。
2. variable_scope 创建的命名空间将对 get_variable 的变量名产生影响 。
3. 创建 named_scope 时，遇到同名的 scope ，系统会自动改名创建一个新的 scope。
4. 创建 named_scope 时，可以通过将名字设置为 None 抹掉前面设定的所有命名空间的名字。

## 创建一个 Variable 背后的故事

### tf.Variable()

In [37]:
tf.reset_default_graph()  # 重置变量



#import tensorflow as tf
a = tf.Variable(1, name='a')
g = tf.get_default_graph()
ops = g.get_operations()
print(ops)
print('------------\n')
for i,op in enumerate(ops):
    print('__________op[%d]:__________'%i)
    print(op)

[<tf.Operation 'a/initial_value' type=Const>, <tf.Operation 'a' type=VariableV2>, <tf.Operation 'a/Assign' type=Assign>, <tf.Operation 'a/read' type=Identity>]
------------

__________op[0]:__________
name: "a/initial_value"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
      }
      int_val: 1
    }
  }
}

__________op[1]:__________
name: "a"
op: "VariableV2"
attr {
  key: "container"
  value {
    s: ""
  }
}
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "shape"
  value {
    shape {
    }
  }
}
attr {
  key: "shared_name"
  value {
    s: ""
  }
}

__________op[2]:__________
name: "a/Assign"
op: "Assign"
input: "a"
input: "a/initial_value"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}
attr {
  key: "_class"
  value {
    list {
      s: "loc:@a"
    }
  }
}
attr {
  key: "use_locking"
  value {
    b: true
  }
}
attr {
  key: "validate_shape"

### tf.get_variable()

上面产生了4个op，这个会产生更多的op

In [4]:
tf.reset_default_graph()  # 重置变量

#import tensorflow as tf
a = tf.get_variable('a',1)
g = tf.get_default_graph()
ops = g.get_operations()
print(ops)
# print('------------\n')

[<tf.Operation 'a/Initializer/random_uniform/shape' type=Const>, <tf.Operation 'a/Initializer/random_uniform/min' type=Const>, <tf.Operation 'a/Initializer/random_uniform/max' type=Const>, <tf.Operation 'a/Initializer/random_uniform/RandomUniform' type=RandomUniform>, <tf.Operation 'a/Initializer/random_uniform/sub' type=Sub>, <tf.Operation 'a/Initializer/random_uniform/mul' type=Mul>, <tf.Operation 'a/Initializer/random_uniform' type=Add>, <tf.Operation 'a' type=VariableV2>, <tf.Operation 'a/Assign' type=Assign>, <tf.Operation 'a/read' type=Identity>]


In [25]:
for i,op in enumerate(ops):
    print('__________op[%d]:__________'%i)
    print(op)

__________op[0]:__________
name: "a/Initializer/random_uniform/shape"
op: "Const"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@a"
    }
  }
}
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
        dim {
          size: 1
        }
      }
      int_val: 1
    }
  }
}

__________op[1]:__________
name: "a/Initializer/random_uniform/min"
op: "Const"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@a"
    }
  }
}
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: -1.7320507764816284
    }
  }
}

__________op[2]:__________
name: "a/Initializer/random_uniform/max"
op: "Const"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@a"
    }
  }
}
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      

## 运算符
原来语句使用`c = a + b`

In [31]:
tf.reset_default_graph()  # 重置变量

#import tensorflow as tf
a = tf.get_variable('a',1)
b = tf.get_variable('b',2)
c = a + b
g = tf.get_default_graph()
ops = g.get_operations()
print (type(ops[-1]))
print('------------')
print (ops[-1])

print('------------')
print('输入： \n',ops[-1].inputs[:])
print('输出： \n',ops[-1].outputs[:])

<class 'tensorflow.python.framework.ops.Operation'>
------------
name: "add"
op: "AddV2"
input: "a/read"
input: "b/read"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}

------------
输入： 
 [<tf.Tensor 'a/read:0' shape=(1,) dtype=float32>, <tf.Tensor 'b/read:0' shape=(2,) dtype=float32>]
输出： 
 [<tf.Tensor 'add:0' shape=(2,) dtype=float32>]


---
使用`c = tf.add(a, b, 'c')` 替换原来的 `c = a + b`

对比输出，发现：

我们发现代码和计算图存在一定的不同，为了编程的方便，一些繁杂的设定被系统以默认值替代，但有时会为我们带来麻烦，如果前面的加法出现错
误（例如参数运算的 Tensor 维度不一致），那么 `add:O`  一定不如 `c:O` 容易定位，所以要尽可能地为每一个 `Op` 起好名字。

In [30]:
tf.reset_default_graph()  # 重置变量

#import tensorflow as tf
a = tf.get_variable('a',1)
b = tf.get_variable('b',2)
c = tf.add(a, b, 'c')  # 替换原来的 c = a + b
g = tf.get_default_graph()
ops = g.get_operations()
print (type(ops[-1]))
print('------------')
print (ops[-1])

print('------------')
print('输入： \n',ops[-1].inputs[:])
print('输出： \n',ops[-1].outputs[:])

<class 'tensorflow.python.framework.ops.Operation'>
------------
name: "c"
op: "Add"
input: "a/read"
input: "b/read"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}

------------
输入： 
 [<tf.Tensor 'a/read:0' shape=(1,) dtype=float32>, <tf.Tensor 'b/read:0' shape=(2,) dtype=float32>]
输出： 
 [<tf.Tensor 'c:0' shape=(2,) dtype=float32>]


## tf. gradients-不懂--------

讲反向计算的一些原理特性和源码分析-没搞懂2020.1.27

In [50]:
tf.reset_default_graph()  # 重置变量


#import tensorflow as tf
a = tf.constant(1.0)
b = tf.constant(1.0)
c = a + b
g = tf.get_default_graph()

ops1 = g.get_operations()
print (ops1)

print('------------')

grad = tf.gradients(c, [a,b])
ops2 = g.get_operations()
print (ops2)

[<tf.Operation 'Const' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'add' type=AddV2>]
------------
[<tf.Operation 'Const' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'add' type=AddV2>, <tf.Operation 'gradients/Shape' type=Const>, <tf.Operation 'gradients/grad_ys_0' type=Const>, <tf.Operation 'gradients/Fill' type=Fill>]


In [51]:
sPath = %pwd
sPath = sPath + r"\logs"
print('保存路径 : ',sPath)

tf.summary.FileWriter(sPath, g).close()  # windows保存要用绝对路径

保存路径 :  X:\0_temp\0_jys\0_我的学习研究\0_机器学习-强化学习\0_learn_pythonLib\learn_Reinforcement_1\logs


最后在cmd窗口下键入下面的代码，利用TensorBoard 显示绘制的信息：
```
tensorboard --logdir=sPath
```
其中`sPath`就是保存的路径

## Optimizer-不懂--------


In [55]:
tf.reset_default_graph()  # 重置变量

#import tensorflow as tf
a = tf.Variable(1.0 ,name='a')
loss = a - 1
opt = tf.train.GradientDescentOptimizer(0.1)
g = tf.get_default_graph()
print(g.get_operations())
print('------------')

grad = opt.compute_gradients(loss)
print(g.get_operations())
print('------------')

train_op = opt.apply_gradients(grad)
print(g.get_operations())
print('------------')

sPath = %pwd
sPath = sPath + r"\logs"
print('保存路径 : ',sPath)
tf.summary.FileWriter(sPath, g).close()

[<tf.Operation 'a/initial_value' type=Const>, <tf.Operation 'a' type=VariableV2>, <tf.Operation 'a/Assign' type=Assign>, <tf.Operation 'a/read' type=Identity>, <tf.Operation 'sub/y' type=Const>, <tf.Operation 'sub' type=Sub>]
------------
[<tf.Operation 'a/initial_value' type=Const>, <tf.Operation 'a' type=VariableV2>, <tf.Operation 'a/Assign' type=Assign>, <tf.Operation 'a/read' type=Identity>, <tf.Operation 'sub/y' type=Const>, <tf.Operation 'sub' type=Sub>, <tf.Operation 'gradients/Shape' type=Const>, <tf.Operation 'gradients/grad_ys_0' type=Const>, <tf.Operation 'gradients/Fill' type=Fill>, <tf.Operation 'gradients/sub_grad/Neg' type=Neg>, <tf.Operation 'gradients/sub_grad/tuple/group_deps' type=NoOp>, <tf.Operation 'gradients/sub_grad/tuple/control_dependency' type=Identity>, <tf.Operation 'gradients/sub_grad/tuple/control_dependency_1' type=Identity>]
------------
[<tf.Operation 'a/initial_value' type=Const>, <tf.Operation 'a' type=VariableV2>, <tf.Operation 'a/Assign' type=Assig

## TensorFlow 的反向传播技巧-不懂--------

In [56]:
import tensorflow as tf
import numpy as np
import sys

In [62]:
def wrong1():
    # --------------------------
    tf.reset_default_graph()  # 重置变量
    # --------------------------
    with tf.name_scope('normal'):
        x = tf.placeholder(tf.float32, [],'x')
        a = tf.Variable(0.0, 'a')
        c = tf.random_uniform([], 0.0, 1.0)
        op = a.assign(c)
        with tf.control_dependencies([op]):
            f = a * x
            g = tf.gradients(f, x)
    with tf.Session() as s:
        s.run(tf.global_variables_initializer())
        print (s.run([f,g], feed_dict={x:1}))



In [61]:
def wrong2():
    # --------------------------
    tf.reset_default_graph()  # 重置变量
    # --------------------------
    with tf.name_scope('normal'):
        x = tf.placeholder(tf.float32, [],'a')
        a = tf.Variable(0.0, 'w')
        c = tf.random_uniform([], 0.0, 1.0)
        op = a.assign(c)
        c2 = tf.random_uniform([], 0.0, 1.0)
        op2 = a.assign(c2)
        with tf.control_dependencies([op]):
            f = a * x
            with tf.control_dependencies([op2]):
                g = tf.gradients(f, x)
    with tf.Session() as s:
        s.run(tf.global_variables_initializer())
        print( s.run([f, g], feed_dict={x: 1}) )

In [63]:
def right1():
    # --------------------------
    tf.reset_default_graph()  # 重置变量
    # --------------------------
    with tf.name_scope('normal'):
        x = tf.placeholder(tf.float32, [],'a')
        a1 = tf.random_uniform([], 0.0, 1.0)
        a2 = tf.random_uniform([], 0.0, 1.0)
        f1 = a1 * x
        f2 = a2 * x
        f = f2 + tf.stop_gradient(f1 - f2)
        g = tf.gradients(f, x)
    with tf.Session() as s:
        s.run(tf.global_variables_initializer())
        print (s.run([f, g], feed_dict={x: 1}))



In [64]:
def right2():
    # --------------------------
    tf.reset_default_graph()  # 重置变量
    # --------------------------
    @tf.RegisterGradient("mult_grad")
    def _mult_grad(op, grad):
        c2 = np.random.uniform(0.0, 1.0)
        return  op.inputs[1] * grad, c2 * grad

    g = tf.get_default_graph()
    x = tf.placeholder(tf.float32, [])
    a = tf.random_uniform([], 0.0, 1.0)
    with g.gradient_override_map({"Mul":"mult_grad"}):
        f = tf.multiply(a, x)
        g = tf.gradients(f, x)
    with tf.Session() as s:
        s.run(tf.global_variables_initializer())
        print (s.run([f, g], feed_dict={x: 1}))

In [65]:
wrong1()

[0.1357274, [0.1357274]]


In [66]:
wrong2()

[0.25120497, [0.25120497]]


In [67]:
right1()

[0.62621486, [0.3169099]]


In [68]:
right2()

[0.6795342, [0.18168582]]


## arg_scope的使用


In [70]:
# --------------------------
tf.reset_default_graph()  # 重置变量
# --------------------------

#import tensorflow as tf

add_arg_scope = tf.contrib.framework.add_arg_scope
arg_scope = tf.contrib.framework.arg_scope


@add_arg_scope
def func1(*args, **kwargs):
    return (args, kwargs)

with arg_scope((func1,), a=1, b=None, c=[1]):
    args, kwargs = func1(0)
    print(args)
    print(kwargs)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

(0,)
{'a': 1, 'b': None, 'c': [1]}


# TensorFlow 的分布式训练

## 基于 MPI 的数据并行模型

1. Scatter
2. Gather
3. Allgather
4. MPI_Bcast
5. MPI_Reduce
6. MPI_Allreduce

## MPI 的实现： mpi_adam

# 基于 TensorFlow 实现经典网络结构

本节将展示基于 TensorFlow 的一些经典网络实现，这些网络包括:
- 多层感知器（ Multi Layer Perceptron, MLP ）;
- 卷积神经网络（ Convolutional Neural Network, CNN ）;
- 循环神经网络（ Recurrent Neural Network ,RNN）。


## 多层感知机

多层感知器主要由全连接层和非线性激活层组成，全连接层一般由矩阵乘法表示，输入的向量 $z$ 和参数矩阵 $W$ 相乘后，得到的结果和向量 $b$ 相加，就得到了结果 $z$ ，计算过程如下所示：

$$z=Wx+b$$

由于输出中的每一个值要经过所有输入值的计算，因此这个结构被称为全连接层 。非线性激活层一般由线性整流函数（ Rectified Linear Unit, ReLU ）表示，它可以**将输入的负数部分变为 0 ，这样模型便满足了一定的非线性效果**。 对于一个 3 层网络组成的感知器。

![IMG20200128092501.png](.\img\IMG20200128092501.png)

In [2]:
# --------------------------
tf.reset_default_graph()  # 重置变量
# --------------------------

#import tensorflow as tf

# 网络参数
n_hidden_1 = 256 # 第1层网络神经元数量
n_hidden_2 = 256 # 第2层网络神经元数量
n_input = 784 # 数据输入维度
n_classes = 10 # 数据输出维度

# placeholder
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# 模型中的参数定义
weights = {
    'h1' : tf.Variable( tf.random_normal([n_input,    n_hidden_1]) ),
    'h2' : tf.Variable( tf.random_normal([n_hidden_1, n_hidden_2]) ),
    'out': tf.Variable( tf.random_normal([n_hidden_2, n_classes] ) )
}
biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden_1])),
    'b2' : tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

def multilayer_perceptron(x):
    # 第一层全连接层和非线性层，下同
    layer_1 = tf.nn.relu( tf.add(tf.matmul(x,       weights['h1']), biases['b1']) )
    layer_2 = tf.nn.relu( tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']) )
    out     =                    tf.matmul(layer_2, weights['out']) + biases['out']
    return out

## 卷积神经网络

卷积神经网络的卷积层可 以在提取局部信息的同时尽可能地减少计算量 。VGG Net 是卷积神经网络曾经的代表作， 它在 网络结构设计上拥有自 己的特点，其中一些设计思路也被很多网络沿用至今。

![IMG20200128092708.png](.\img\IMG20200128092708.png)

整个模型分为 6 个部分，前 5 个部分由卷积层组成， 最后一个部分由全连接层组成。 

在每一个卷积部分内， 图像的尺度并没有发生变化， 只有在进入下一个部分时，图像的长宽依次减半，但与此同时，它的通道数也扩大了一倍，这就相当于图像的信息在不断地集中变少， 而对图像分析的方式在不断增多 。 

VGG Net的很多设计也被后来的模型使用 。

卷积神经网络的世界还有很多优秀的模型结构 ， 例如**残差网络**等，希望读者能够自行学习了解 。

In [4]:
# --------------------------
tf.reset_default_graph()  # 重置变量
# --------------------------

#import tensorflow as tf

# 5个卷积部分的输出通道数
filter_depth = [64, 128, 256, 512] 
VGG16_NUM_HIDDEN_1, VGG16_NUM_HIDDEN_2 = 4096, 1000

def submodel(x, iters, k_width, in_depth, out_depth):
    for i in range(iters):
        cur_in_depth = in_depth if i == 0 else out_depth
        w = tf.Variable(tf.truncated_normal([k_width, k_width, cur_in_depth, out_depth], stddev=0.01))
        b = tf.Variable(tf.zeros([out_depth]))
        x = tf.nn.relu(tf.nn.conv2d(x, w, [1,1,1,1], padding='SAME') + b)
    x = tf.nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    return x

def fc(x, in_size, out_size):
    w = tf.Variable(tf.truncated_normal([in_size, out_size]))
    b = tf.Variable(tf.constant(0.0, shape=[out_size]))
    out = tf.matmul(x, w) + b
    return tf.nn.relu(out)
  
def vgg16(x, filter_depth, hidden1, hidden2, img_height, img_width, img_depth, num_labels):
    out = x
    k_w = 3
    # 1st part
    k_d1 = filter_depth[0]
    out = submodel(out, 2, k_w, img_depth, k_d1)
    # 2nd part
    k_d2 = filter_depth[1]
    out = submodel(out, 2, k_w, k_d1, k_d2)
    # 3rd part
    k_d3 = filter_depth[2]
    out = submodel(out, 3, k_w, k_d2, k_d3)
    # 4th part
    k_d4 = filter_depth[3]
    out = submodel(out, 3, k_w, k_d3, k_d4)
    # 5th part
    out = submodel(out, 3, k_w, k_d4, k_d4)

    element_num = ((img_height // (2 ** 5)) * (img_width // (2 ** 5)) * k_d4)
    out = tf.reshape(out, [-1, element_num])

    # fully connected part
    out = fc(out, element_num, VGG16_NUM_HIDDEN_1)
    out = tf.nn.dropout(out, 0.5)
    out = fc(out, VGG16_NUM_HIDDEN_1, VGG16_NUM_HIDDEN_2)
    out = tf.nn.dropout(out, 0.5)

    return fc(out, VGG16_NUM_HIDDEN_2, num_labels)

## 循环神经网络

在前面介绍的模型中，我们可以通过传入某个固定的输入得到固定的输出 ，模型并不能对输入数据产生记忆 。 当我们遇到一些序列预测问题时＼这种基于时间或位置关系的问题又需要考虑时间或空间上的依赖关系，因此前面提到的模型不能满足这样的需求，于是我们就需要使用循环神经网络。 



循环神经网络实现了对序列数据的建模和拟合，当我们使用模型计算出某一输入数据的结构后，部分信息就被记录到模型中，这些信息会在下一个输入计算时产生作用，这些记录的信息被称为隐含层的状态信息 。

模型的整体结构如图 4-24 所示，其中 $x$ 表示模型的输入， $h$ 表示模型的隐含状态， 。$o$表示每一时刻的输出，而 $U 、 V$ 和 $w$表示模型的参数。 图 4-24 中左边展示的是聚合版本的模型，经过展开后可以得到右边的样子 ， 这样可以清晰地看出时刻之间的依赖关系 。


![IMG20200128145840.png](.\img\IMG20200128145840.png)


经典的 时仆J 模型在训练时存在梯度消失的问题 ，为了解决这个问题，前人发明了很多改进版的模型，而 LSTM ( Long Short Term Memory ）是其中最常见的模型，它能很好地解决因为时间跨度长而导致的梯度消失问题，它的基本结构如图 4-25 所示 。


![IMG20200128150300.png](.\img\IMG20200128150300.png)

In [5]:
# --------------------------
tf.reset_default_graph()  # 重置变量
# --------------------------

#import tensorflow as tf

# 网络参数
num_input = 28 # 数据输入的维度
timesteps = 28 # 数据输入的总时长
num_hidden = 128 # 隐含层的维度
num_classes = 10 # 算法最终的输出维度

# placeholder
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# 定义LSTM之外的全连接层的参数
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

def RNN(x, weights, biases):
    x = tf.unstack(x, timesteps, 1)
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    return tf.matmul(outputs[-1], weights['out']) + biases['out']

# 总结
本章介绍了 TensorFlow 的基本使用方法和一些常见的与训练相关的内容，让我们回顾一下。

1. TensorFlow 的使用主要包括：计算图定义、操作运算地点安排和计算图执行三部分。
2. TensorFlow 内部的一些运行机理很值得读者做一定的研究 。
3. 使用 MPI 的通信方式可以使 TensorFlow 完成分布式训练。
4. TensorFlow 可以实现常见的各种经典模型 。